In [1]:
import modern_robotics as mr
import numpy as np
import math

In [6]:
def ik(px,py,pz):
    # constants
    L1 = 15
    L2 = 105
    L3 = (55.95+57.75)
    L4 = 12.31

    # swtich to r1 home frame
    x_e = px * 1000
    y_e = py * 1000 + 98
    z_e = pz * 1000 - 100
    # theta3
    d = math.sqrt(x_e**2 + y_e**2 + z_e**2 - L1**2)
    beta = math.acos((L3**2 + L4**2 + L2**2 - d**2)/(2 * L2 * math.sqrt(L3**2 + L4**2)))
    eta = math.atan(L4 / L3)
    delta = math.pi - beta
    # How do we tell the difference?
    theta3_elbow_down = math.pi - beta + eta
    theta3_elbow_up = math.pi + beta + eta
    theta3_elbow_up = theta3_elbow_up - 2 * math.pi if theta3_elbow_up >= 2 * math.pi - 1e-5 else theta3_elbow_up

    # theta2
    EB = math.sqrt(x_e**2 + z_e**2)
    EC = math.sqrt(L3**2+L4**2) * math.sin(delta)
    d = math.sqrt(EB**2 - EC**2)
    m = math.sqrt(x_e**2 + y_e**2 + z_e**2 - EC**2)
    phi = math.acos(abs(y_e)/m)
    alpha = math.acos(L1 / m)
    theta2 = -(alpha - phi) # direction of theta2 is opposite

    # theta1
    CK = math.sqrt(m**2 - L1**2)
    DK = L1 * math.tan(theta2)  # When theta2 is negative, DK will be negative, too
    DC = CK + DK
    BO = DC*math.cos(theta2)
    AO = math.sqrt(x_e**2 + z_e**2)
    epsilon = math.acos(BO/AO)
    gamma = math.atan(-z_e/x_e)
    theta1_elbow_up = -epsilon + gamma
    theta1_elbow_down = epsilon + gamma # direction of theta1 is opposite

    return [theta1_elbow_down, theta2, theta3_elbow_down]

In [7]:
def PoE_3D(theta1, theta2, theta3):
    M = np.array([
        [1, 0, 0, (105 + 55.95 + 57.75)/1000],
        [0, 1, 0, (-98 + (-15))/1000],
        [0, 0, 1, (100 - 12.31)/1000],
        [0, 0, 0, 1]
    ])
    omega_1 = np.array([0,1,0])
    omega_2 = np.array([0,0,1])
    omega_3 = np.array([0,-1,0])

    q_1 = np.array([0,-98,100])/1000
    q_2 = np.array([0,-98,100])/1000
    q_3 = np.array([105,-113,100])/1000

    v_1 = - np.cross(omega_1,q_1)
    v_2 = - np.cross(omega_2,q_2) 
    v_3 = - np.cross(omega_3,q_3) 
    
    S1 = mr.MatrixExp6(mr.VecTose3(np.concatenate((omega_1,v_1),axis=0))*theta1)
    S2 = mr.MatrixExp6(mr.VecTose3(np.concatenate((omega_2,v_2),axis=0))*theta2)
    S3 = mr.MatrixExp6(mr.VecTose3(np.concatenate((omega_3,v_3),axis=0))*theta3)

    T_05 = S1.dot(S2).dot(S3).dot(M)
    return T_05

In [8]:
xyz = [[0.2165269048192407, 0.21652207556751119, 0.21648946064209207, 0.21640447921183573, 0.21624591086301734, 0.21599574624744927, 0.2156390377305957, 0.21516375003968707, 0.2145606109118349, 0.213822961742146, 0.21294660823183725, 0.21192967103634996, 0.21077243641346466, 0.20947720687141538, 0.20804815181700434, 0.20649115820371658, 0.20481368117983428, 0.20302459473655152, 0.20113404235608862, 0.19915328765980694, 0.19709456505632325, 0.1949709303896242, 0.19279611158718118, 0.1905843593080645, 0.18835029759105815, 0.18610877450277435, 0.18387471278576803, 0.18166296050665132, 0.1794881417042083, 0.17736450703750928, 0.17530578443402553, 0.17332502973774386, 0.17143447735728098, 0.1696453909139982, 0.16796791389011592, 0.16641092027682813, 0.16498186522241717, 0.16368663568036781, 0.16252940105748248, 0.1615124638619952, 0.16063611035168643, 0.15989846118199755, 0.15929532205414537, 0.15882003436323686, 0.15846332584638326, 0.15821316123081514, 0.1580545928819967, 0.15796961145174043, 0.1579369965263214, 0.1579321672745918, 0.1579321672745918, 0.1579241650320374, 0.15787012093603328, 0.15772930367165752, 0.15746655025564846, 0.1570520185549974, 0.1564609398055418, 0.15567337113055774, 0.15467394805935325, 0.15345163704586084, 0.1519994879872306, 0.1503143867424228, 0.1483968076508012, 0.14625056605072556, 0.1438825707981446, 0.14130257678518907, 0.13852293745876434, 0.13555835733914354, 0.1324256445385602, 0.12914346327980136, 0.12573208641480035, 0.12221314794322957, 0.11860939553109352, 0.11494444302932164, 0.11124252299236116, 0.1075282391967699, 0.10382631915980942, 0.10016136665803749, 0.09655761424590147, 0.0930386757743307, 0.08962729890932966, 0.08634511765057082, 0.08321240484998751, 0.08024782473036668, 0.07746818540394199, 0.0748881913909864, 0.07252019613840556, 0.0703739545383298, 0.06845637544670817, 0.06677127420190035, 0.06531912514327014, 0.06409681412977775, 0.06309739105857322, 0.06230982238358931, 0.061718743634133655, 0.06130421193348262, 0.06104145851747341, 0.06090064125309775, 0.0608465971570938, 0.06083859491453924, 0.06083859491453924, 0.06083937805352699, 0.0608447648143419, 0.06085909961409288, 0.06088644941131717, 0.060930609547213424, 0.06099510958687485, 0.061083219160522215, 0.061197953804736985, 0.061342080803694424, 0.06151812503039665, 0.061728374787905715, 0.06197488765057673, 0.06225949630529094, 0.06258381439268879, 0.06294924234840302, 0.06335697324429179, 0.06380799862967171, 0.06430311437255096, 0.06484292650086239, 0.06542785704369657, 0.06605814987253492, 0.06673387654248275, 0.0674549421335024, 0.06822109109164627, 0.06903191307028995, 0.06988684877136535, 0.07078519578659362, 0.07172611443871847, 0.07270863362273908, 0.07373165664714322, 0.07479396707514041, 0.07589423456589499, 0.07703102071575908, 0.07820278489950586, 0.07940789011156252, 0.08064460880724342, 0.0819111287439831, 0.08320555882256947, 0.08452593492837682, 0.08587022577259895, 0.08723633873348219, 0.08862212569755862, 0.090025388900879, 0.09144388677024598, 0.09287533976444712, 0.09431743621548799, 0.09576783816982531, 0.09722418722959997, 0.09868411039387007, 0.10014522589984416, 0.10160514906411425, 0.10306149812388889, 0.10451190007822621, 0.10595399652926706, 0.10738544952346825, 0.10880394739283522, 0.11020721059615561, 0.111592997560232, 0.11295911052111528, 0.1143034013653374, 0.11562377747114475, 0.11691820754973109, 0.11818472748647077, 0.1194214461821517, 0.12062655139420833, 0.12179831557795508, 0.12293510172781924, 0.12403536921857379, 0.125097679646571, 0.12612070267097514, 0.12710322185499565, 0.12804414050712062, 0.12894248752234885, 0.12979742322342425, 0.13060824520206799, 0.13137439416021182, 0.13209545975123146, 0.1327711864211793, 0.13340147925001763, 0.13398640979285184, 0.1345262219211632, 0.13502133766404253, 0.13547236304942242, 0.13588009394531125, 0.13624552190102537, 0.13656983998842329, 0.1368544486431375, 0.1371009615058085, 0.1373112112633175, 0.1374872554900198, 0.13763138248897724, 0.13774611713319193, 0.13783422670683937, 0.13789872674650072, 0.13794288688239711, 0.13797023667962147, 0.13798457147937226, 0.13798995824018737, 0.13799074137917497], [-0.13206076875311867, -0.1320705292590678, -0.13213644799584395, -0.13230820581714636, -0.13262869176376343, -0.13313430492192432, -0.13385525628165074, -0.13481587059510827, -0.13603488823495816, -0.13752576705270872, -0.13929698423706716, -0.14135233817229098, -0.14369125029653965, -0.14630906696022614, -0.14919736128436853, -0.15234423501894168, -0.15573462040122876, -0.15935058201417274, -0.16317161864472815, -0.1671749651422126, -0.17133589427665835, -0.17562801859716387, -0.18002359229024553, -0.18449381303818913, -0.18900912387740146, -0.19353951505676203, -0.19805482589597434, -0.20252504664391796, -0.20692062033699962, -0.2112127446575051, -0.21537367379195088, -0.21937702028943534, -0.22319805691999073, -0.2268140185329347, -0.23020440391522176, -0.23335127764979496, -0.23623957197393725, -0.23885738863762385, -0.24119630076187254, -0.24325165469709645, -0.24502287188145483, -0.2465137506992054, -0.2477327683390553, -0.24869338265251267, -0.24941433401223911, -0.24991994717040003, -0.2502404331170172, -0.2504121909383195, -0.2504781096750955, -0.2504878701810448, -0.2504878701810448, -0.2504927135935172, -0.25052542415519646, -0.2506106547752351, -0.2507696880916612, -0.25102058626120594, -0.2513783407491312, -0.2518550221190575, -0.252459929822792, -0.25319974199015555, -0.2540786652188114, -0.2550985843640921, -0.2562592123288278, -0.2575582398531739, -0.2589914853044385, -0.26055304446691074, -0.2622354403316881, -0.2640297728865043, -0.2659258689055571, -0.267912431739336, -0.26997719110445006, -0.2721070528734557, -0.2742882488646844, -0.2765064866320702, -0.2787470992549783, -0.28099519512803167, -0.28323580775093965, -0.2854540455183256, -0.2876352415095542, -0.28976510327855987, -0.29182986264367394, -0.29381642547745285, -0.2957125214965056, -0.2975068540513218, -0.29918924991609913, -0.30075080907857143, -0.30218405452983604, -0.3034830820541821, -0.3046437100189179, -0.30566362916419865, -0.3065425523928544, -0.307282364560218, -0.3078872722639525, -0.30836395363387875, -0.308721708121804, -0.3089726062913487, -0.3091316396077749, -0.3092168702278135, -0.3092495807894926, -0.30925442420196514, -0.30925442420196514, -0.30925382768739307, -0.3092497246081538, -0.3092388058350211, -0.30921797357811465, -0.3091843369376498, -0.30913520745468914, -0.30906809466189233, -0.3089807016342672, -0.30887092053992027, -0.30873682819080706, -0.3085766815934833, -0.3083889134998548, -0.3081721279579289, -0.3079250958625644, -0.30764675050622253, -0.30733618312971767, -0.3069926384729674, -0.30661551032574397, -0.3062043370784241, -0.30575879727274013, -0.3052787051525306, -0.30476400621449046, -0.30421477275892217, -0.3036311994404861, -0.3030135988189511, -0.30236239690994543, -0.30167812873570676, -0.30096143387583363, -0.3002130520180353, -0.2994338185088827, -0.2986246599045592, -0.2977865895216111, -0.2969207029876979, -0.29602817379234364, -0.29511024883768694, -0.2941682439892317, -0.29320353962659806, -0.2922175761942726, -0.29121184975235914, -0.2901879075273296, -0.28914734346277404, -0.2880917937701519, -0.28702293247954225, -0.28594246699039444, -0.2848521336222789, -0.2837536931656376, -0.2826489264325347, -0.2815396298074072, -0.28042761079781553, -0.27931468358519435, -0.2782026645756027, -0.27709336795047523, -0.27598860121737234, -0.27489016076073103, -0.2737998273926155, -0.2727193619034677, -0.271650500612858, -0.2705949509202359, -0.26955438685568034, -0.26853044463065073, -0.26752471818873735, -0.26653875475641187, -0.26557405039377824, -0.264632045545323, -0.2637141205906663, -0.262821591395312, -0.26195570486139885, -0.2611176344784507, -0.2603084758741272, -0.25952924236497465, -0.25878086050717636, -0.2580641656473031, -0.2573798974730645, -0.2567286955640588, -0.2561110949425238, -0.25552752162408776, -0.2549782881685195, -0.2544635892304793, -0.25398349711026974, -0.25353795730458584, -0.25312678405726596, -0.25274965591004245, -0.25240611125329226, -0.2520955438767873, -0.2518171985204456, -0.251570166425081, -0.2513533808831551, -0.25116561278952665, -0.2510054661922029, -0.2508713738430896, -0.25076159274874266, -0.25067419972111765, -0.2506070869283208, -0.2505579574453602, -0.2505243208048953, -0.2505034885479887, -0.2504925697748562, -0.2504884666956167, -0.2504878701810448], [0.08270090974292187, 0.08270133224770151, 0.08270418568393743, 0.08271162059569508, 0.08272549352861071, 0.08274738009648824, 0.08277858804789606, 0.08282017033276398, 0.08287293816897998, 0.08293747410898705, 0.08301414510638022, 0.08310311558250319, 0.08320436049304537, 0.08331767839463855, 0.08344270451145394, 0.08357892380179888, 0.08372568402471378, 0.08388220880656896, 0.0840476107076614, 0.08422090428881178, 0.08440101917796118, 0.084586813136768, 0.08477708512720479, 0.08497058837815513, 0.08516604345201045, 0.0853621513112669, 0.0855576063851222, 0.08575110963607255, 0.08594138162650933, 0.08612717558531616, 0.08630729047446556, 0.08648058405561594, 0.08664598595670839, 0.08680251073856354, 0.08694927096147846, 0.08708549025182341, 0.08721051636863879, 0.08732383427023198, 0.08742507918077413, 0.08751404965689713, 0.08759072065429029, 0.08765525659429738, 0.08770802443051334, 0.08774960671538128, 0.08778081466678911, 0.08780270123466663, 0.08781657416758228, 0.0878240090793399, 0.08782686251557582, 0.08782728502035547, 0.08782728502035547, 0.08782798512586176, 0.08783271337159265, 0.08784503328585425, 0.08786802123105776, 0.08790428805553684, 0.08795600074536529, 0.08802490407617453, 0.08811234226497118, 0.08821928062195465, 0.08834632720233464, 0.08849375445814875, 0.08866152089007998, 0.08884929269927433, 0.08905646543915836, 0.08928218566725672, 0.08952537259700973, 0.08978473974959089, 0.09005881660572446, 0.09034597025750311, 0.09064442706020531, 0.090952294284113, 0.0912675817663291, 0.09158822356259512, 0.09191209959910862, 0.0922370573243409, 0.09256093336085441, 0.09288157515712042, 0.09319686263933655, 0.09350472986324422, 0.09380318666594642, 0.09409034031772506, 0.09436441717385866, 0.09462378432643981, 0.0948669712561928, 0.09509269148429116, 0.09529986422417519, 0.09548763603336956, 0.09565540246530081, 0.0958028297211149, 0.0959298763014949, 0.09603681465847837, 0.09612425284727502, 0.09619315617808424, 0.0962448688679127, 0.09628113569239179, 0.09630412363759529, 0.09631644355185691, 0.09632117179758774, 0.09632187190309406, 0.09632187190309406, 0.09632111975413152, 0.09631594615580259, 0.09630217860632607, 0.0962759110830262, 0.09623349843224609, 0.09617155075926134, 0.09608692781819367, 0.09597673340192435, 0.09583830973200794, 0.09566923184858575, 0.09546730200029942, 0.09523054403420453, 0.0949571977856842, 0.09464571346836252, 0.09429474606401829, 0.09390314971249854, 0.09346997210163197, 0.09299444885714275, 0.09247599793256395, 0.09191421399915106, 0.09130886283579574, 0.09065987571893923, 0.08996734381248597, 0.08923151255771723, 0.08845277606320459, 0.08763167149472359, 0.08676887346516723, 0.0858651884244596, 0.08492154904946947, 0.08393900863392373, 0.08291873547832111, 0.08186200727984569, 0.08077020552228048, 0.07964480986592098, 0.07848739253748878, 0.07729961272004506, 0.07608321094290423, 0.07484000347154754, 0.07357187669753651, 0.07228078152842665, 0.07096872777768097, 0.0696377785545835, 0.06829004465415293, 0.0669276789470562, 0.06555287076952201, 0.06416784031325438, 0.06277483301534634, 0.06137611394819331, 0.0599739622094069, 0.05857066531172829, 0.05716851357294189, 0.055769794505788894, 0.05437678720788082, 0.052991756751613245, 0.05161694857407899, 0.050254582866982256, 0.04890684896655171, 0.04757589974345426, 0.04626384599270855, 0.0449727508235987, 0.04370462404958769, 0.04246141657823102, 0.0412450148010902, 0.04005723498364643, 0.03889981765521425, 0.037774421998854765, 0.0366826202412895, 0.035625892042814106, 0.03460561888721149, 0.03362307847166576, 0.032679439096675686, 0.03177575405596794, 0.030912956026411635, 0.030091851457930623, 0.029313114963417965, 0.028577283708649252, 0.02788475180219599, 0.027235764685339458, 0.02663041352198415, 0.026068629588571254, 0.0255501786639925, 0.025074655419503204, 0.02464147780863668, 0.02424988145711685, 0.02389891405277274, 0.023587429735450992, 0.023314083486930634, 0.023077325520835797, 0.022875395672549547, 0.022706317789127253, 0.022567894119210855, 0.022457699702941616, 0.022373076761873865, 0.02231112908888919, 0.02226871643810896, 0.022242448914809015, 0.02222868136533268, 0.02222350776700357, 0.022222755618041157]]



In [15]:
new_xyz = [[], [], []]
for x, y, z in zip(xyz[0], xyz[1], xyz[2]):
    t1, t2, t3 = ik(x, y, z)
    t3 += np.deg2rad(20)
    nx, ny, nz = PoE_3D(t1, t2, t3)[:3,3]
    new_xyz[0].append(nx)
    new_xyz[1].append(ny)
    new_xyz[2].append(nz)

In [16]:
np.rad2deg(ik(new_xyz[0][0], new_xyz[1][0], new_xyz[2][0]))

array([ 5., -5., 27.])

In [17]:
new_xyz

[[0.21252703321977,
  0.21245339820985315,
  0.2121304864252072,
  0.21161542178254852,
  0.2109465848468987,
  0.21013271354337404,
  0.20917589534490622,
  0.20807755019099095,
  0.2068402328165003,
  0.20546827340646456,
  0.20396799732176202,
  0.20234775295284185,
  0.2006178314587822,
  0.19879031215091741,
  0.19687884792521598,
  0.19489839766427602,
  0.19286491051256083,
  0.19079496784684719,
  0.18870539141739154,
  0.18661282976867194,
  0.18453333900821786,
  0.18248197759970014,
  0.18047243739779734,
  0.17851673393341422,
  0.17662497746250777,
  0.1748052422559388,
  0.17306354516628267,
  0.17140393622003613,
  0.16982869477143236,
  0.16833861574011907,
  0.16693336274432796,
  0.16561185940299628,
  0.16437268719504938,
  0.1632144580776872,
  0.16213613225476017,
  0.16113725550240804,
  0.1602180956953851,
  0.15937966410614574,
  0.15862361329083494,
  0.15795200969527973,
  0.15736698534676796,
  0.15687027894303146,
  0.15646268191593,
  0.1561434087603487,
  